# Decumulate rain fields

and write it back to GRIB, ensuring proper validity

In [ ]:
%matplotlib inline
# for figures in notebook

# import & initialize epygram
import epygram
epygram.init_env()

import os
INPUTS_DIR = os.path.join('..', 'inputs')
OUTPUTS_DIR = os.path.join('..', 'outputs')

In [ ]:
paths = [os.path.join(INPUTS_DIR, 'grid.arome-forecast.guyane0025+00{:02}:00.grib'.format(i))
         for i in (0, 12, 24)]
resources = [epygram.formats.resource(p, 'r') for p in paths]

In [ ]:
rain = [r.readfield({'discipline':0, 'parameterCategory':1, 'parameterNumber':65})
        for r in resources[1:]]  # start from 1: term 0, no rain

In [ ]:
# rain is accumulated from beginning of forecast
print([r.mean() for r in rain])

In [ ]:
for i in range(len(rain)):
    if i == 0: continue  # term 1, no decumulation to do
    rain[i].operation('-', rain[i-1])
    rain[i].validity[0].set(cumulativeduration=(rain[i].validity[0].get()-rain[i-1].validity[0].get()))

In [ ]:
print([r.mean() for r in rain])

In [ ]:
rain_series = epygram.formats.resource(os.path.join(OUTPUTS_DIR,'rain_series.grib'), 'w', fmt='GRIB')
for r in rain:
    print(r.validity[0])
    rain_series.writefield(r)
rain_series.close()